In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW
from sklearn.model_selection import train_test_split
import numpy as np

# Импортируем дополнительные библиотеки
import pandas as pd
from seqeval.metrics import f1_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# Шаг 1: Загрузка и подготовка данных
# Предположим, у вас есть данные в формате DataFrame с колонками 'sentence' и 'labels'

# Пример загрузки данных из файла CSV
# df = pd.read_csv('ner_dataset.csv')

# Здесь мы создадим пример данных
data = [
    {'sentence': 'Джон Смит работает в компании OpenAI в Сан-Франциско.', 
     'labels': ['B-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'O', 'B-LOC', 'O']},
    {'sentence': 'Мария поехала в Москву.', 
     'labels': ['B-PER', 'O', 'O', 'B-LOC', 'O']},
    # Добавьте больше примеров данных
]

df = pd.DataFrame(data)

# Шаг 2: Создание списков предложений и меток
sentences = df['sentence'].tolist()
labels = df['labels'].tolist()

# Шаг 3: Загрузка токенизатора и модели
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Создаем словарь меток
unique_tags = set(tag for doc in labels for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}



In [ ]:
# Шаг 4: Подготовка данных для модели
class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len, tag2id):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.tag2id = tag2id

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        word_labels = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            sentence,
            is_split_into_words=False,
            add_special_tokens=True,
            max_length=self.max_len,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        labels = np.ones(self.max_len, dtype=int) * -100  # Инициализируем метки как -100

        offsets = encoding['offset_mapping'][0].numpy()
        input_ids = encoding['input_ids'][0]

        label_index = 0
        for idx, (start, end) in enumerate(offsets):
            if start == end:
                continue  # Идентификатор без позиции в исходном тексте (например, специальные токены)
            if list(self.tokenizer.decode([input_ids[idx]]))[0] == ' ':
                continue  # Пропускаем пробельные символы

            if label_index < len(word_labels):
                labels[idx] = self.tag2id[word_labels[label_index]]
                label_index += 1

        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(labels)

        return item

# Шаг 5: Разбиение данных на обучающую и валидационную выборки
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# Шаг 6: Создание датасетов и загрузчиков данных
max_len = 50
batch_size = 16

train_dataset = NERDataset(train_sentences, train_labels, tokenizer, max_len, tag2id)
val_dataset = NERDataset(val_sentences, val_labels, tokenizer, max_len, tag2id)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Шаг 7: Инициализация модели
num_labels = len(tag2id)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

# Шаг 8: Определение оптимизатора
optimizer = AdamW(model.parameters(), lr=5e-5)

# Шаг 9: Определение устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



In [ ]:
# Шаг 10: Создание класса BERTNERTrainer (если он еще не создан)
# Предположим, что вы уже определили класс BERTNERTrainer с методами __init__, fit, eval и train

# Вот пример минимальной реализации класса BERTNERTrainer:

class BERTNERTrainer:
    def __init__(self, model, optimizer, train_loader, valid_loader, tokenizer, tag2id, id2tag, device, epochs=3, model_save_path='best_model.bin', max_len=50):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.tokenizer = tokenizer
        self.tag2id = tag2id
        self.id2tag = id2tag
        self.device = device
        self.epochs = epochs
        self.model_save_path = model_save_path
        self.max_len = max_len

    def fit(self):
        self.model.train()
        losses = []
        true_labels = []
        pred_labels = []

        for data in self.train_loader:
            input_ids = data['input_ids'].to(self.device)
            attention_mask = data['attention_mask'].to(self.device)
            labels = data['labels'].to(self.device)

            self.optimizer.zero_grad()

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            self.optimizer.step()

            losses.append(loss.item())

            preds = torch.argmax(logits, dim=2)
            preds = preds.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            for i in range(len(labels)):
                pred_tags = []
                true_tags = []
                for j in range(len(labels[i])):
                    if labels[i][j] != -100:
                        pred_tags.append(self.id2tag[preds[i][j]])
                        true_tags.append(self.id2tag[labels[i][j]])
                pred_labels.append(pred_tags)
                true_labels.append(true_tags)

        avg_loss = np.mean(losses)
        f1 = f1_score(true_labels, pred_labels)
        return f1, avg_loss

    def eval(self):
        self.model.eval()
        losses = []
        true_labels = []
        pred_labels = []

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                labels = data["labels"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                logits = outputs.logits

                losses.append(loss.item())

                preds = torch.argmax(logits, dim=2)
                preds = preds.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()

                for i in range(len(labels)):
                    pred_tags = []
                    true_tags = []
                    for j in range(len(labels[i])):
                        if labels[i][j] != -100:
                            pred_tags.append(self.id2tag[preds[i][j]])
                            true_tags.append(self.id2tag[labels[i][j]])
                    pred_labels.append(pred_tags)
                    true_labels.append(true_tags)

        avg_loss = np.mean(losses)
        f1 = f1_score(true_labels, pred_labels)
        report = classification_report(true_labels, pred_labels)
        print("Validation Loss: {:.4f}".format(avg_loss))
        print("Validation F1-Score: {:.4f}".format(f1))
        print(report)
        return f1, avg_loss

    def train(self):
        best_f1 = 0.0
        for epoch in range(self.epochs):
                        print(f'Epoch {epoch + 1}/{self.epochs}')
            
            train_f1, train_loss = self.fit()
            print(f'Train Loss: {train_loss:.4f} F1-Score: {train_f1:.4f}')

            val_f1, val_loss = self.eval()
            print(f'Validation Loss: {val_loss:.4f} F1-Score: {val_f1:.4f}')
            print('-' * 50)

            if val_f1 > best_f1:
                torch.save(self.model.state_dict(), self.model_save_path)
                best_f1 = val_f1

        # Загрузка наилучшей модели после обучения
        self.model.load_state_dict(torch.load(self.model_save_path))
        self.model.to(self.device)

    def predict(self, text):
        # Шаг 1: Токенизация входного текста
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        # Шаг 2: Получение предсказаний модели
        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

        # Шаг 3: Преобразование логитов в предсказанные метки
        logits = outputs.logits  # или outputs[0], в зависимости от модели
        predictions = torch.argmax(logits, dim=2)

        # Перенос на CPU и преобразование в numpy
        predictions = predictions.cpu().numpy()[0]
        input_ids = input_ids.cpu().numpy()[0]

        # Шаг 4: Преобразование идентификаторов токенов в сами токены
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids)

        # Шаг 5: Преобразование идентификаторов меток в названия меток
        predicted_labels = [self.id2tag[pred] for pred in predictions]

        # Шаг 6: Обработка токенов и меток, исключая специальные токены
        final_tokens = []
        final_labels = []

        for token, label in zip(tokens, predicted_labels):
            if token not in self.tokenizer.all_special_tokens:
                final_tokens.append(token)
                final_labels.append(label)

        # Шаг 7: Возвращение токенов и соответствующих им предсказанных меток
        return list(zip(final_tokens, final_labels))

# Шаг 11: Создание объекта trainer
trainer = BERTNERTrainer(
    model=model,
    optimizer=optimizer,
    train_loader=train_loader,
    valid_loader=val_loader,
    tokenizer=tokenizer,
    tag2id=tag2id,
    id2tag=id2tag,
    device=device,
    epochs=3,  # Вы можете изменить количество эпох
    model_save_path='best_ner_model.bin',
    max_len=max_len
)

# Шаг 12: Обучение модели
trainer.train()

# После обучения вы можете использовать метод predict для предсказания
test_sentence = "Иван Иванович посетил Нью-Йорк в прошлом году."
predictions = trainer.predict(test_sentence)
print(predictions)